# Exponential Integrators for Parabolic Equations

Exponential integrators combine low order time-stepping methods to obtain high order in time.

We consider the discretized linear homogeneous parabolic equation:

$$
M \frac{d}{dt} u + A u = 0 \qquad \text{on} \; [0,T]
$$

with $u(0) = u_0$. For the general case see  M. Hochbruck and A. Ostermann, *Exponential integrators*, Acta Numerical, p209-286, 2010.


First, we perform $m$ steps of the implicit Euler method with time-step $\tau = \frac{T}{m}$:

$$
u_{i+1} = u_i - \tau (M + \tau A)^{-1} A u_i \qquad 0 \leq i < m
$$

We define the low dimensional space $V_m$ (of dimensions m+1) spanned by all time-steps:

$$
V_m = \text{span} \{ u_0, \ldots u_m \}
$$

The idea is that the true solution of the ODE can be well approximated in $V_m$. Thus, in the second step we perform a Galerkin discretization of the ODE in $V_m$:

Find $u^m : [0,T] \rightarrow V_m$ such that

$$
v^T M \frac{d u^m}{dt} + v^T A u^m = 0 \qquad \forall \, v \in V_m
$$

For implementation, we put the $m+1$ basis vectors $u_i$ as column vectors into the representation matrix 

$$
V_m = \Big( u_0, u_1, \ldots u_m \Big)
$$

It is a high rectangular matrix of dimension $N \times (m+1)$, where $N$ is the dimension of the FE - space.

Every $u \in V_m$ can be written as $V_m y$, with a short vector $y \in {\mathbb R}^{m+1}$, and test-functions $v = V_m z$. Thus, the Galerkin-solution in $V_m$ can be obtained via

Find $y : [0,T] \rightarrow {\mathbb R}^{m+1}$:

$$
z^T V_m^T M V_m \frac{d y}{dt} + V_m^T A V_m y = 0 \qquad \forall \, z \in {\mathbb R}^{m+1}
$$

Since $u_0$ is in the space $V_m$, the initial condition can be represented as $u_0 = V_m y_0$.


In short:

$$
M^s \frac{d y}{dt} + A^s y = 0
$$

with the small matrices

\begin{eqnarray*}
M^s & = & V_m^T M V_m \\
A^s & = & V_m^T A V_m \\
\end{eqnarray*}

This low-dimensional ODE can be solved exactly by its eigensystem.

For implementation, there is a stability issue: Since all sample vectors $u_i$ are similar, the matrix $V_m$ is very ill-conditioned.
Thus, the sample vectors must be orthogonalized (by Gram-Schmidt or QR - decomposition).


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep
from scipy.linalg import eigh

mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

In [2]:
u0 = exp(-100*( (x-0.5)**2 + (y-0.5)**2))

fes = H1(mesh, order=4)
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

In [3]:
def ExponentialPropagation (mform, aform, gfu, T, steps):
    tau = T/steps
    m = BilinearForm(mform).Assemble()
    a = BilinearForm(aform).Assemble()
    mstar = BilinearForm(mform+tau*aform).Assemble()
    mstarinv = mstar.mat.Inverse()
    un = gfu.vec.CreateVector(copy=True)
    
    vecs = MultiVector(un, 0)
    vecs.Append (un)
    for j in range(steps):
        un.data += tau * mstarinv * (-a.mat * un)
        vecs.Append (un)
    
    vecs.Orthogonalize(m.mat)
    vecs.Orthogonalize(m.mat) # improves orthogonality

    asmall = InnerProduct(vecs, a.mat * vecs)
    msmall = InnerProduct(vecs, m.mat * vecs) # identity
    
    # print ("msmall =", msmall)
    # print ("asmall =", asmall)
    
    lams,evec = eigh(asmall)
    evec = Matrix(evec)
    # print (lams, evec)
    
    mu0 = (m.mat*gfu.vec).Evaluate()
    # u0s = InnerProduct(vecs, mu0)    # available since Feb 10, 2022
    u0s = Vector([InnerProduct(mu0,v) for v in vecs])
    u0s = evec.T * u0s
    uTs = Vector([u0*exp(-lam*T) for u0,lam in zip(u0s,lams)])
    uTs = evec * uTs
    gfu.vec.data = vecs * uTs

In [4]:
gfu = GridFunction(fes)

gfu.Set(u0)
Draw (gfu)

ExponentialPropagation (mform, aform, gfu, 0.02, 8)
Draw (gfu);

WebGuiWidget(value={'ngsolve_version': '6.2.2203-55-ge6a1832fc', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

WebGuiWidget(value={'ngsolve_version': '6.2.2203-55-ge6a1832fc', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

Convergence plot:

In [6]:
Tend = 0.02
mmax = 20
gfuref = GridFunction(fes)
gfuref.Set(u0)
ExponentialPropagation (mform, aform, gfuref, Tend, mmax)

err = []
gfu = GridFunction(fes)
for m in range(1,mmax):
    gfu.Set(u0)
    ExponentialPropagation (mform, aform, gfu, Tend, m)
    err.append (Norm(gfu.vec-gfuref.vec))

print (err)

[0.20600328199376758, 0.06978025266715498, 0.03049776255121646, 0.011407488793323194, 0.0035849111823151934, 0.0007281408027218528, 0.00016400656081982313, 0.0002207732294693555, 0.00010127534375465337, 2.2105692323236415e-05, 6.378919613280206e-06, 7.240522273706592e-07, 7.612651755831167e-07, 4.143020583810261e-07, 1.8219274917452472e-07, 3.968077666128608e-08, 9.11208822466509e-09, 3.34534586041024e-09, 1.4499449472635008e-09]


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.yscale('log')
plt.xlabel("steps")
plt.ylabel("error")
plt.plot(err, "-*")